###### Bulgaria Employment Trends 
[Go back to the main page](./../Bulgaria_Employment_Trends.ipynb)

---------

## Offers By Job Location

-------
### Introduction


### Methodology



In [1]:
import psycopg2
import requests
import pandas as pd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True) # run at the start of every ipython notebook to use plotly.offline

%matplotlib notebook
%matplotlib inline

In [2]:
conn = psycopg2.connect("dbname=jobsbg")
counts_df = pd.read_sql_query('SELECT * FROM v_offers_count_per_location', conn)

In [3]:
counts_df.head()

,job_location,offer_count
0,София,273986
1,Варна,61557
2,Пловдив,55716
3,Бургас,24116
4,Русе,16893


### Geocoding Locations with OpenStreetMap's Nominatim

In [4]:
# query = 'place Бяла, Варна'
# 
# url_template = 'https://nominatim.openstreetmap.org/search?format=json&limit=1&q={}'
# url = url_template.format(query)
# response = requests.get(url, timeout=20)
# results = response.json()
# 
# results[0]['lat'], results[0]['lon']

### Join Locations by Name and Fix Discrepancies

In [5]:
places_locs_df = pd.read_csv('../data/bg_settl_locations.csv')

locs_df = places_locs_df.set_index('name').join(counts_df.set_index('job_location'), how='left', rsuffix='l')
locs_df = locs_df[locs_df.offer_count.notna()]
locs_df = locs_df.sort_values(by='offer_count', ascending=False)
locs_df.head()

,latitude,longitude,offer_count
София,42.6972,23.3243,273986.0
Варна,43.2063,27.9252,61557.0
Пловдив,42.1359,24.7491,55716.0
Бургас,42.4915,27.4718,24116.0
Русе,43.8351,25.9576,16893.0


In [6]:
def hoverbox_builder(loc):
    """Populates a text template to prepare a hoverbox from hovertext."""
    return f"\
<b>{loc.index}</b><br>\
{job.Index.strftime('%d %b %Y')}<br>\
{job.company_name}<br>\
<b>{job.text_salary}</b> ➡ <i>(norm. {job.norm_salary:.0f} BGN)</i>"





locs_df['text'] = locs_df.index + '<br>Offers count: ' + locs_df['offer_count'].astype(int).astype(str)


locs_df.head()

,latitude,longitude,offer_count,text
София,42.6972,23.3243,273986.0,София<br>Offers count: 273986
Варна,43.2063,27.9252,61557.0,Варна<br>Offers count: 61557
Пловдив,42.1359,24.7491,55716.0,Пловдив<br>Offers count: 55716
Бургас,42.4915,27.4718,24116.0,Бургас<br>Offers count: 24116
Русе,43.8351,25.9576,16893.0,Русе<br>Offers count: 16893


In [16]:
data = []
scale = 200

df_sub = locs_df[0:50]

for i in range(len(df_sub)):
    city = dict(
        type = 'scattergeo',
        lon = df_sub.longitude,
        lat = df_sub.latitude,
        text = df_sub.text,
        marker = dict(
            size = df_sub['offer_count'] / scale,
            #sizeref  = 2. * max(df_sub['pop']/scale) / (25*2),
            color = '#99d8c9',
            line = dict(width=1, color='#99d8c9'),
            sizemode = 'area',
        ),
        name = 'all_cities')
    
    data.append(city)


In [17]:
layout = dict(
    paper_bgcolor = '#fff',            
    plot_bgcolor = '#fff',
    title = 'Number of Job Offers by Location',
    showlegend = False,
    margin=dict( l=0, r=0, b=0, t=50, pad=4, autoexpand=True),
    geo = dict(
        projection=dict(type='mercator',),
        resolution=50, 
        lonaxis = dict(range=[21.5, 30.0]),
        lataxis = dict(range=[41.2, 44.4]),
        showframe = False,
        showland = True,
        showcountries = True,
        countrywidth=2,
        bgcolor = 'rgba(255, 255, 255, 0.0)',
        landcolor = 'rgba(150, 150, 150, 0.2)',
        countrycolor="rgb(255, 255, 255)",
        coastlinecolor = "rgb(255, 255, 255)",
        ),
    )

In [18]:
fig = dict(data=data, layout=layout)

plotly.offline.iplot(fig, filename = 'offers_by_location_map.html')

In [10]:


# plotly.offline.iplot(fig, filename = 'offers_over_time.html')

# Uncomment the line below to export an HTML version of the chart.
# plotly.offline.plot(fig, filename = 'offers_over_time-daily_result.html')